In [ ]:
# ------------------------------------------------------------
# EXPERIMENT: Transfer Learning using Pre-trained VGG16 Model
# ------------------------------------------------------------

import os
import numpy as np
from PIL import Image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# 1️⃣ Define dataset path
base_dir = r'C:\Users\Acer\Desktop\lp4'
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')

# 2️⃣ Clean and recreate dummy folders
import shutil
if os.path.exists(train_dir):
    shutil.rmtree(train_dir)
if os.path.exists(valid_dir):
    shutil.rmtree(valid_dir)

for folder in ['train/cats', 'train/dogs', 'valid/cats', 'valid/dogs']:
    os.makedirs(os.path.join(base_dir, folder), exist_ok=True)

# 3️⃣ Generate dummy images
for subdir in ['train/cats', 'train/dogs', 'valid/cats', 'valid/dogs']:
    for i in range(10):
        arr = np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8)
        Image.fromarray(arr).save(os.path.join(base_dir, subdir, f'{i}.jpg'))

print("✅ Dummy dataset created successfully at:", base_dir)

# 4️⃣ Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=8, class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    valid_dir, target_size=(224, 224), batch_size=8, class_mode='categorical')

# ✅ Automatically get number of classes from generator
num_classes = len(train_generator.class_indices)
print(f"Detected {num_classes} classes: {list(train_generator.class_indices.keys())}")

# 5️⃣ Load base VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

# 6️⃣ Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# 7️⃣ Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🔹 Training only new classifier layers...\n")
model.fit(train_generator, epochs=5, validation_data=validation_generator)

# 🔟 Fine-tuning (unfreeze)
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("\n🔹 Fine-tuning the entire model...\n")
model.fit(train_generator, epochs=10, initial_epoch=5, validation_data=validation_generator)

# 1️⃣1️⃣ Save model
model.save(os.path.join(base_dir, "vgg16_finetuned.h5"))
print("\n✅ Model training complete and saved as 'vgg16_finetuned.h5'")


✅ Dummy dataset created successfully at: C:\Users\Acer\Desktop\lp4
Found 20 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Detected 2 classes: ['cats', 'dogs']

🔹 Training only new classifier layers...

Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 28s 9s/step - accuracy: 0.5000 - loss: 0.7391 - val_accuracy: 0.5000 - val_loss: 0.7039
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 9s/step - accuracy: 0.5000 - loss: 0.7036 - val_accuracy: 0.5000 - val_loss: 0.7005
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 22s 9s/step - accuracy: 0.5000 - loss: 0.6994 - val_accuracy: 0.5000 - val_loss: 0.6951
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 41s 9s/step - accuracy: 0.5000 - loss: 0.6982 - val_accuracy: 0.5000 - val_loss: 0.7006
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 22s 9s/step - accuracy: 0.5000 - loss: 0.6987 - val_accuracy: 0.5000 - val_loss: 0.7023

🔹 Fine-tuning the entire model...

Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 65s 19s/step - accuracy: 0.5000 - loss: 0.6979 - val_accuracy: 0.5000 - val_loss: 0.69

In [33]:
import os
print(os.getcwd())


C:\Users\Acer\Desktop\lp4
